# Sandbox for running and testing individual tasks

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=1

## Count and plot objects in images

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountAnalysisTask
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

imgs = get_all_files_from_paths("/home/juliocesar/leaf-detection/imgs/first_analysis", stem_sort=True)
analysis = DetectionAnalysisContext(
    imgs = imgs, 
    config="/home/juliocesar/leaf-detection/object-detection-analysis/analysis_config.yaml", 
    tasks=[
        ("ObjectCount", CountAnalysisTask(plot_all=True, plot_per_class=False)),
    ],
)
analysis.run()

## Validate model count (error in getting true count of objects)

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountValidateModelTask
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

TEST_IMG_PATH = "/home/juliocesar/leaf-detection/datasets/leaf-dataset/test/images"
TEST_LABELS = "/home/juliocesar/leaf-detection/datasets/leaf-dataset/test/labels"
imgs = get_all_files_from_paths(TEST_IMG_PATH)
analysis = DetectionAnalysisContext(
    imgs=imgs, 
    config="/home/juliocesar/leaf-detection/object-detection-analysis/analysis_config.yaml", 
    tasks=[
        ("CountValidation", CountValidateModelTask(annotations_path=TEST_LABELS)),
    ]
)
analysis.change_configs({"detection_parameters": {"slice_detect": False, "detection_filters": {"duplicate_filter": False, "object_size_filter": False}}})
analysis.run()

In [3]:
analysis.export_detections()

## Object color classification

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountAnalysisTask, ObjectClassificationTask, ObjectAreaOccupationTask
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

imgs = [
    "/home/juliocesar/leaf-detection/imgs/analysis_test/0_640x640.png",
    "/home/juliocesar/leaf-detection/imgs/analysis_test/1_640x640.png",
    "/home/juliocesar/leaf-detection/imgs/analysis_test/2_640x640_aglom.png",
    "/home/juliocesar/leaf-detection/imgs/old_scg_control_tests/set25_14.png",
]

analysis = DetectionAnalysisContext(
    imgs=imgs, 
    config="/home/juliocesar/leaf-detection/object-detection-analysis/analysis_config.yaml", 
    tasks=[
        ("ObjectClassification", ObjectClassificationTask(
            clf="resnet18_mlp", 
            clf_ckpt_path="/home/juliocesar/leaf-detection/checkpoints/classifiers/mlp_rn18.pt", 
            clf_cls_labels=["dark", "dead", "light", "medium"], 
            clf_cls_colors={"dark": "red", "dead": "black", "light": "deepskyblue", "medium": "yellow"},
            show_detections=True, 
            plot_per_image=True,
        )),
    ]
)
analysis.run()


## Object area occupation

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountAnalysisTask, ObjectClassificationTask, ObjectAreaOccupationTask
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

# FATAL: currently there is no way to segment a big image without overheading memory.
# TODO: fix this memory overhead (implement on-slice segmentation)

# imgs = [
#     "/home/juliocesar/leaf-detection/imgs/analysis_test/0_640x640.png",
#     "/home/juliocesar/leaf-detection/imgs/analysis_test/1_640x640.png",
#     "/home/juliocesar/leaf-detection/imgs/analysis_test/2_640x640_aglom.png",
#     "/home/juliocesar/leaf-detection/imgs/old_scg_control_tests/set25_14.png",
# ]
imgs = get_all_files_from_paths("/home/juliocesar/leaf-detection/imgs/first_analysis", stem_sort=True)[:1]
analysis = DetectionAnalysisContext(
    imgs=imgs, 
    config="/home/juliocesar/leaf-detection/object-detection-analysis/analysis_config.yaml", 
    tasks=[
        ("ObjectAreaOccupation", ObjectAreaOccupationTask(plot_per_class=True)),
    ]
)
analysis.run()


## Custom pipeline

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=1

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountAnalysisTask, ObjectClassificationTask, ObjectAreaOccupationTask
from object_detection_analysis.classifiers import classifier_from_name
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

IMG_DIR = "/home/juliocesar/leaf-detection/imgs/oct1_analysis"
imgs = get_all_files_from_paths(IMG_DIR, stem_sort=True, skip_ext=[".txt"])

clf = "mlp"
clf_path = "/home/juliocesar/leaf-detection/checkpoints/classifiers/mlp_rn34.pt"
clf_cls_labels = ["dark", "dead", "light", "medium"]
clf_cls_colors = {"dark": "red", "dead": "black", "light": "deepskyblue", "medium": "#B024C0"}

analysis = DetectionAnalysisContext(
    imgs=imgs,
    config="analysis_config.yaml",
    tasks=[
        ("LeafCount", CountAnalysisTask(plot=True, xlabel="Image ID", ylabel="Number of leaves")),
        ("LeafAreaOccupation", ObjectAreaOccupationTask(use_boxes=False, plot=True, xlabel="Image ID", ylabel="Area ratio occupied by leaves")),
        ("LeafColorClassification", ObjectClassificationTask(clf=clf, clf_cls_colors=clf_cls_colors, clf_cls_labels=clf_cls_labels, clf_input_batch=16, clf_ckpt_path=clf_path, show_classifications=False, show_detections=False, use_boxes=False, plot=True, xlabel="Image ID", ylabel="Number of leaves per classification")), 
    ],
    image_specific_parameters = {
        2: { "slice_wh": (512,512) },
        9: { "slice_wh": (512,512) },
    },
)
analysis.run()
analysis.task_summary()
analysis.plot_tasks(plot_total_results=False, plot_per_class_results=True)
analysis.export_csv_tasks()

In [3]:
analysis.export_detections()

# Test

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountAnalysisTask, ObjectClassificationTask, ObjectAreaOccupationTask
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

imgs = [
    "/home/juliocesar/leaf-detection/imgs/analysis_test/0_640x640.png",
    "/home/juliocesar/leaf-detection/imgs/analysis_test/1_640x640.png",
    "/home/juliocesar/leaf-detection/imgs/analysis_test/2_640x640_aglom.png",
    "/home/juliocesar/leaf-detection/imgs/old_scg_control_tests/set25_14.png",
]
# imgs = get_all_files_from_paths("/home/juliocesar/leaf-detection/imgs/first_analysis", stem_sort=True)

analysis = DetectionAnalysisContext(
    imgs=imgs, 
    config="/home/juliocesar/leaf-detection/object-detection-analysis/analysis_config.yaml", 
    tasks=[
        ("ObjectCount", CountAnalysisTask(plot=True, xlabel="Image ID", ylabel="Number of objects")),
        ("ObjectAreaOccupation", ObjectAreaOccupationTask(use_boxes=False, plot=True, xlabel="Image ID", ylabel="Area occupied by objects")),
        ("ObjectClassification", ObjectClassificationTask(
            clf="cnn_fc", 
            clf_ckpt_path="/home/juliocesar/leaf-detection/checkpoints/classifiers/cnn.pt", 
            clf_cls_labels=["dark", "dead", "light", "medium"], 
            clf_cls_colors={"dark": "red", "dead": "black", "light": "deepskyblue", "medium": "yellow"},
            show_classifications=False,
            show_detections=False, 
            use_boxes=False,
            plot=True, xlabel="Image ID", ylabel="Number of objects per classified class",
        )),
    ],
)
analysis.change_configs({"detection_parameters": {"slice_detect": False}})
analysis.run()


In [ ]:
analysis.plot_tasks(plot_total_results=False, plot_per_class_results=True, save_plots=False)
analysis.export_csv_tasks(skip_tasks=None)

In [ ]:
from object_detection_analysis.analysis import DetectionAnalysisContext
from object_detection_analysis.tasks import CountValidateModelTask
from scg_detection_tools.utils.file_handling import get_all_files_from_paths

TEST_IMG_PATH = "/home/juliocesar/leaf-detection/datasets/gen-leaf-dataset-v9/test/images"
TEST_LABELS = "/home/juliocesar/leaf-detection/datasets/gen-leaf-dataset-v9/test/labels"
imgs = get_all_files_from_paths(TEST_IMG_PATH)
analysis = DetectionAnalysisContext(
    imgs=imgs, 
    config="/home/juliocesar/leaf-detection/object-detection-analysis/analysis_config.yaml", 
    tasks=[
        ("CountValidation", CountValidateModelTask(annotations_path=TEST_LABELS)),
    ]
)
results = analysis.run()

In [ ]:
# Test new image masking code
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from object_detection_analysis.analysis import DetectionAnalysisContext

imgs = [
    "/home/juliocesar/leaf-detection/imgs/objdet_test/0_640x640.png",
]
analysis = DetectionAnalysisContext(imgs=imgs, config="analysis_config.yaml")
analysis.change_configs({"detection_parameters": {"slice_detect": False}})
analysis._run_detections()
analysis._run_segmentations()
analysis.show_masks()
analysis.show_detections()